# Multi-Stage Job Advertisement Analysis — BERT Zone Identification Model Evaluation

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mansamoussa/llm-skill-extractor/blob/main/notebooks/03_evaluation.ipynb)

---